In [ ]:
from pygame.locals import *
import random
import sys
import os
import pygame
import datetime

# Initialize Pygame
pygame.init()
pygame.mixer.init()


save_file_path = "progress_save.txt"

# fonts 
font = pygame.font.Font(None, 36)

# Load sounds
def load_sound(path):
    try:
        return pygame.mixer.Sound(path)
    except pygame.error as e:
        print(f"Error loading sound: {path}, {e}")
        return None

def load_music(path):
    try:
        pygame.mixer.music.load(path)
        pygame.mixer.music.play(-1)
    except pygame.error as e:
        print(f"Error loading music: {path}, {e}")


crash_sound = pygame.mixer.Sound(r'Rush rider\\collision.mp3')
powerup_sound = pygame.mixer.Sound(r'Rush rider\\level-up.mp3')
pygame.mixer.music.load(r'Rush rider\\background_music.mp3')
game_over_voice = load_sound(r'Rush rider\\Game_over.mp3')

pygame.mixer.music.play(-1)

# Screen settings
WIDTH, HEIGHT = 500, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Rush Rider')

# Game state
speed = 2
score = 0
fuel = 100
selected_car_index = 0
gameover = False
is_day = True
camera_shake_duration = 0
in_shop = False
in_start_screen = True

# Lanes and markers
marker_width = 10
marker_height = 50
left_lane = 150
center_lane = 250
right_lane = 350
lanes = [left_lane, center_lane, right_lane]
lane_marker_move_y = 0

# Road and edge markers
road = (100, 0, 300, HEIGHT)
left_edge_marker = (95, 0, marker_width, HEIGHT)
right_edge_marker = (395, 0, marker_width, HEIGHT)

# Day/Night Themes
DAY_COLORS = {
    'background': (76, 208, 56),
    'road': (100, 100, 100),
    'lane': (255, 255, 255),
    'edge': (255, 232, 0)
}
NIGHT_COLORS = {
    'background': (20, 20, 20),
    'road': (40, 40, 40),
    'lane': (150, 150, 150),
    'edge': (200, 200, 0)
}

# Load car images
car_image_paths = [
  
    r"Rush rider\\car2.png",
    r"Rush rider\\car3.png",
]
car_images = [pygame.transform.scale(pygame.image.load(path).convert_alpha(), (60, 120)) for path in car_image_paths]

# Power-ups
POWER_UPS = ['speed_boost', 'fuel_refill']
power_up_image_paths = {
    'speed_boost': r"Rush rider\speed_boost.png",
    'fuel_refill': r"Rush rider\fuel refill.png"
}
power_up_images = {
    name: pygame.transform.scale(pygame.image.load(path).convert_alpha(), (40, 40))
    for name, path in power_up_image_paths.items()
}

# Vehicle Classes
class Vehicle(pygame.sprite.Sprite):
    def __init__(self, image, x, y, size=(90, 120)):
        super().__init__()
        self.image = pygame.transform.scale(image, size)
        self.rect = self.image.get_rect()
        self.rect.center = [x, y]

class PlayerVehicle(Vehicle):
    def __init__(self, image, x, y):
        super().__init__(image, x, y, size=(60, 120))

# Load enemy images
enemy_image_files = [ 'semi trailer.png', 'taxi.png', 'van.png', 'Enemy car.png']
base_path = r'Rush rider'
enemy_images = [pygame.transform.scale(pygame.image.load(os.path.join(base_path, name)).convert_alpha(), (100, 120)) for name in enemy_image_files]

# Crash image
try:
    crash_image = pygame.image.load(os.path.join(base_path, 'crash car.png')).convert_alpha()
    crash_rect = crash_image.get_rect()
except pygame.error as e:
    print(f"Error loading crash image: {e}")
    crash_image = pygame.Surface((60, 120))
    crash_image.fill((255, 0, 0))
    crash_rect = crash_image.get_rect()

# Clock
clock = pygame.time.Clock()
fps = 60
running = True

# Sprites
player_group = pygame.sprite.Group()
vehicle_group = pygame.sprite.Group()
power_up_group = pygame.sprite.Group()

# Initialize player here to avoid None error
player = None

while running:
    clock.tick(fps)

    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            sys.exit()

        if event.type == KEYDOWN:
            if in_start_screen:
                in_start_screen = False
                in_shop = True
        
            
            elif in_shop:
                if event.key == K_LEFT:
                    selected_car_index = (selected_car_index - 1) % len(car_images)
                elif event.key == K_RIGHT:
                    selected_car_index = (selected_car_index + 1) % len(car_images)
                elif event.key == K_RETURN:
                    in_shop = False
                    gameover = False
                    speed = 2
                    score = 0
                    fuel = 100
                    player = PlayerVehicle(car_images[selected_car_index], 250, 400)
                    player_group.empty()
                    player_group.add(player)
                    vehicle_group.empty()
                    power_up_group.empty()

            elif not gameover and player:
                if event.key == K_LEFT and player.rect.center[0] > left_lane:
                    player.rect.x -= 100
                elif event.key == K_RIGHT and player.rect.center[0] < right_lane:
                    player.rect.x += 100
                elif event.key == K_d:
                    is_day = not is_day

            elif gameover:
                if event.key == K_y:
                    in_start_screen = True
                elif event.key == K_n:
                    pygame.quit()
                    sys.exit()

    # Save progress on game over
  
    with open(save_file_path, "w") as f:
        f.write(f"Score: {score}\n")
        f.write(f"Fuel: {int(fuel)}%\n")
        f.write(f"Speed: {speed}\n")

    theme = DAY_COLORS if is_day else NIGHT_COLORS

    if in_start_screen:
        screen.fill((0, 0, 0))
        font = pygame.font.Font(None, 48)
        title = font.render("RUSH RIDER", True, (255, 255, 255))
        title_rect = title.get_rect(center=(WIDTH // 2, HEIGHT // 2 - 50))
        screen.blit(title, title_rect)

        font_small = pygame.font.Font(None, 32)
        prompt = font_small.render("Press any key to start", True, (200, 200, 200))
        prompt_rect = prompt.get_rect(center=(WIDTH // 2, HEIGHT // 2 + 30))
        screen.blit(prompt, prompt_rect)

        pygame.display.update()
        continue

    elif in_shop:
        screen.fill((30, 30, 30))
        font = pygame.font.Font(None, 36)
        text = font.render("Select Your Car", True, (255, 255, 255))
        screen.blit(text, (WIDTH // 2 - text.get_width() // 2, 30))

        for i, car_img in enumerate(car_images):
            x = 100 + i * 120
            y = HEIGHT // 2 - 60
            screen.blit(car_img, (x, y))
            if i == selected_car_index:
                pygame.draw.rect(screen, (255, 0, 0), (x, y, 60, 120), 3)

    elif not gameover:
        screen.fill(theme['background'])
        pygame.draw.rect(screen, theme['road'], road)
        pygame.draw.rect(screen, theme['edge'], left_edge_marker)
        pygame.draw.rect(screen, theme['edge'], right_edge_marker)

        lane_marker_move_y += speed * 2
        if lane_marker_move_y >= marker_height * 2:
            lane_marker_move_y = 0
        for y in range(-marker_height * 2, HEIGHT, marker_height * 2):
            pygame.draw.rect(screen, theme['lane'], (left_lane + 45, y + lane_marker_move_y, marker_width, marker_height))
            pygame.draw.rect(screen, theme['lane'], (center_lane + 45, y + lane_marker_move_y, marker_width, marker_height))

        if len(vehicle_group) < 2:
            add_vehicle = all(vehicle.rect.top > vehicle.rect.height * 1.5 for vehicle in vehicle_group)
            if add_vehicle:
                lane = random.choice(lanes)
                img = random.choice(enemy_images)
                vehicle = Vehicle(img, lane, -200)
                vehicle_group.add(vehicle)

        if random.randint(0, 100) < 2:
            power_up_type = random.choice(POWER_UPS)
            x_pos = random.choice(lanes)
            power_up = pygame.sprite.Sprite()
            power_up.image = power_up_images[power_up_type]
            power_up.rect = power_up.image.get_rect()
            power_up.rect.center = (x_pos, -50)
            power_up.type = power_up_type
            power_up_group.add(power_up)

        for vehicle in vehicle_group:
            vehicle.rect.y += speed
            if vehicle.rect.top >= HEIGHT:
                vehicle.kill()
                score += 1
                if score % 5 == 0:
                    speed += 1

        for power_up in power_up_group:
            power_up.rect.y += speed
            if power_up.rect.top >= HEIGHT:
                power_up.kill()

        collected_power_ups = pygame.sprite.spritecollide(player, power_up_group, True)
        for power_up in collected_power_ups:
            powerup_sound.play()
            if power_up.type == 'fuel_refill':
                fuel = min(100, fuel + 25)
            elif power_up.type == 'speed_boost':
                speed += 1

        if pygame.sprite.spritecollide(player, vehicle_group, True):
            crash_sound.play()
            gameover = True
            crash_rect.center = player.rect.center
            camera_shake_duration = 20
            # play game_over
            if game_over_voice:
                game_over_voice.play()
        
        # Decrease fuel gradually
        fuel -= 0.1
        if fuel < 0:
            fuel = 0
            gameover = True
            crash_rect.center = player.rect.center

        player_group.draw(screen)
        vehicle_group.draw(screen)
        power_up_group.draw(screen)

        # Display HUD with score, speed, fuel
        font = pygame.font.Font(None, 24)
        score_text = font.render(f"Score: {score}", True, (255, 255, 255))
        speed_text = font.render(f"Speed: {speed}%", True, (255, 255, 255))
        fuel_text = font.render(f"Fuel: {int(fuel)}%", True, (255, 255, 255))
        screen.blit(score_text, (10, 10))
        screen.blit(speed_text, (10, 40))
        screen.blit(fuel_text, (10, 70))

        # Fuel bar
        bar_x, bar_y = 100, 70
        bar_width, bar_height = 120, 15
        pygame.draw.rect(screen, (50, 50, 50), (bar_x, bar_y, bar_width, bar_height), border_radius=4)
        fuel_ratio = max(0, min(1, fuel / 100))
        fuel_fill = int(bar_width * fuel_ratio)
        if fuel > 50:
            fuel_color = (0, 255, 0)  # Green
        elif fuel > 20:
            fuel_color = (255, 165, 0)  # Orange
        else:
            fuel_color = (255, 0, 0)  # Red
        pygame.draw.rect(screen, fuel_color, (bar_x, bar_y, fuel_fill, bar_height), border_radius=4)

    else:
        # Game over screen with shake effect
        if camera_shake_duration > 0:
            shake_offset = random.randint(-5, 5)
            screen.blit(crash_image, crash_rect.move(shake_offset, shake_offset))
            camera_shake_duration -= 1
        else:
            screen.blit(crash_image, crash_rect)

        pygame.draw.rect(screen, (200, 0, 0), (0, 50, WIDTH, 100))
        font = pygame.font.Font(None, 24)
        text = font.render('Game Over. Play again? (Y or N)', True, (255, 255, 255))
        text_rect = text.get_rect(center=(WIDTH // 2, 100))
        screen.blit(text, text_rect)

        # HUD panel on game over screen
        hud_height = 70
        hud_surface = pygame.Surface((WIDTH, hud_height), pygame.SRCALPHA)
        hud_surface.fill((0, 0, 0, 150))
        screen.blit(hud_surface, (0, HEIGHT - hud_height))

        font_big = pygame.font.Font(None, 28)
        score_text = font_big.render(f"Score: {score}", True, (255, 255, 0))
        speed_text = font_big.render(f"Speed: {speed}%", True, (255, 100, 100))
        fuel_text = font_big.render(f"Fuel: {int(fuel)}%", True, (255, 255, 255))

        screen.blit(score_text, (20, HEIGHT - hud_height + 10))
        screen.blit(speed_text, (165, HEIGHT - hud_height + 10))
        screen.blit(fuel_text, (WIDTH - 150, HEIGHT - hud_height + 10))

        # Fuel bar
        bar_x = WIDTH - 150
        bar_y = HEIGHT - hud_height + 35
        bar_width = 120
        bar_height = 12
        pygame.draw.rect(screen, (50, 50, 50), (bar_x, bar_y, bar_width, bar_height), border_radius=4)
        fuel_ratio = max(0, min(1, fuel / 100))
        fuel_fill = int(bar_width * fuel_ratio)
        if fuel > 50:
            fuel_color = (0, 255, 0)
        elif fuel > 20:
            fuel_color = (255, 165, 0)
        else:
            fuel_color = (255, 0, 0)
        pygame.draw.rect(screen, fuel_color, (bar_x, bar_y, fuel_fill, bar_height), border_radius=4)
        displayed_score = score
        score_multiplier = 1
     # Display HUD with score, speed, fuel, and daily challenge
        font = pygame.font.Font(None, 24)
        score_text = font.render(f"Score: {displayed_score} (x{score_multiplier})", True, (255, 255, 255))
        speed_text = font.render(f"Speed: {speed}", True, (255, 255, 255))
        fuel_text = font.render(f"Fuel: {int(fuel)}%", True, (255, 255, 255))
      

        screen.blit(score_text, (10, 10))
        screen.blit(speed_text, (10, 35))
        screen.blit(fuel_text, (10, 60))
     
    pygame.display.update()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
